### Vector Strores and retrivers 

The Notebook contain the vector store and retriever abstractions. These abstarction are designed to support retriveal of data- from vector store databases and other sources -- for integration with LLM workflows/ THey are important for application that fetech data to be resoned over as part of model inference, as in the case of retrieval-augmented generation 

Things that are covered--

Documents 

Vector Stores

Retrievers 

#### Documents 

Langchain implements a documents abstraction, which intended to represent a unit of text and associated metadata. It has two attributes 

1.page content- a string respresenting the content of the document

2.metadata- a dict containing arbitary metadata. The metadata attribute can capture information about the source of the data and its relationship to other documents and other information.Note that an individual document object represents a chunk of larger document

In [7]:
from langchain_core.documents import Document


documents = [
    Document(
        page_content="Dogs are great companions, known for their loyaltya and friendliness",
        metadata={"source": "mammal-pets-docs"},
    ),
    Document(
        page_content="Cars are independent pets that often enjoy thier own space",
        metadata={"source": "mammal-pets-docs"},
    ),
    Document(
        page_content="Goldfish are popular pets for begineers, requiring relatively simple care",
        metadata={"source": "fish-pets-docs"},
    ),
    Document(
        page_content="Parots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "mammal-pets-docs"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop arround",
        metadata={"source": "mammal-pets-docs"},
    ),
    
]


In [8]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key= os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000153743B4FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000153743B6210>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [10]:
## vectorstore 
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore 


In [12]:
vectorstore.similarity_search("Cat")

[Document(id='9184aaec-1b89-4c19-b55c-68c1f7f82a60', metadata={'source': 'mammal-pets-docs'}, page_content='Cars are independent pets that often enjoy thier own space'),
 Document(id='842d8bf2-8faa-4724-ad47-d2d8e8bd8b7c', metadata={'source': 'mammal-pets-docs'}, page_content='Dogs are great companions, known for their loyaltya and friendliness'),
 Document(id='14a54b57-fe15-473f-ba9f-8f2cce493f53', metadata={'source': 'mammal-pets-docs'}, page_content='Rabbits are social animals that need plenty of space to hop arround'),
 Document(id='7044ccaa-e6bd-4a0f-bb9d-4eec788d13d4', metadata={'source': 'fish-pets-docs'}, page_content='Goldfish are popular pets for begineers, requiring relatively simple care')]

In [13]:
##async

await vectorstore.asimilarity_search("cat")

[Document(id='9184aaec-1b89-4c19-b55c-68c1f7f82a60', metadata={'source': 'mammal-pets-docs'}, page_content='Cars are independent pets that often enjoy thier own space'),
 Document(id='842d8bf2-8faa-4724-ad47-d2d8e8bd8b7c', metadata={'source': 'mammal-pets-docs'}, page_content='Dogs are great companions, known for their loyaltya and friendliness'),
 Document(id='14a54b57-fe15-473f-ba9f-8f2cce493f53', metadata={'source': 'mammal-pets-docs'}, page_content='Rabbits are social animals that need plenty of space to hop arround'),
 Document(id='7044ccaa-e6bd-4a0f-bb9d-4eec788d13d4', metadata={'source': 'fish-pets-docs'}, page_content='Goldfish are popular pets for begineers, requiring relatively simple care')]

In [17]:
# Perform similarity search
results = vectorstore.similarity_search_with_score("cat")
print(results)


[(Document(id='9184aaec-1b89-4c19-b55c-68c1f7f82a60', metadata={'source': 'mammal-pets-docs'}, page_content='Cars are independent pets that often enjoy thier own space'), 1.395796298980713), (Document(id='842d8bf2-8faa-4724-ad47-d2d8e8bd8b7c', metadata={'source': 'mammal-pets-docs'}, page_content='Dogs are great companions, known for their loyaltya and friendliness'), 1.5535989999771118), (Document(id='14a54b57-fe15-473f-ba9f-8f2cce493f53', metadata={'source': 'mammal-pets-docs'}, page_content='Rabbits are social animals that need plenty of space to hop arround'), 1.5779528617858887), (Document(id='7044ccaa-e6bd-4a0f-bb9d-4eec788d13d4', metadata={'source': 'fish-pets-docs'}, page_content='Goldfish are popular pets for begineers, requiring relatively simple care'), 1.7474007606506348)]


### Retrievers 

Langchain vectorstore do not sub class runnable and so cannot be immediately into Langchain expression language chains

LangChain Retrivers are runnables so they implement a standared set of methods (eg, sychronous and asynchronous and batch opearations) and are designed to be incoporated in LCEL chains


we can create a simple version of this ourselves, without subclassing Retriever. if we choose what method we wish to use retrive documents, we can create a runnable easily. Below we will build one arround the similiarity search method :

In [19]:
from typing import List 

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda


retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

retriever.batch (["cat", "dog"])

[[Document(id='9184aaec-1b89-4c19-b55c-68c1f7f82a60', metadata={'source': 'mammal-pets-docs'}, page_content='Cars are independent pets that often enjoy thier own space')],
 [Document(id='842d8bf2-8faa-4724-ad47-d2d8e8bd8b7c', metadata={'source': 'mammal-pets-docs'}, page_content='Dogs are great companions, known for their loyaltya and friendliness')]]

In [20]:
vectorstore.as_retriever(
    search_type="similarity", search_kawrgs={"k": 1})

retriever.batch(["cat","dogs"])



[[Document(id='9184aaec-1b89-4c19-b55c-68c1f7f82a60', metadata={'source': 'mammal-pets-docs'}, page_content='Cars are independent pets that often enjoy thier own space')],
 [Document(id='842d8bf2-8faa-4724-ad47-d2d8e8bd8b7c', metadata={'source': 'mammal-pets-docs'}, page_content='Dogs are great companions, known for their loyaltya and friendliness')]]

In [ ]:
# RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Anwer this question using the provideed context only.

{question}

Context:

{context}
"""

prompt = ChatPromptTemplate.from_messages(["human", message])

rag_chain={"context": retriever,"question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("tell me about dogs?") 
print(response)


content='According to the provided context, dogs are great companions, known for their loyalty and friendliness.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 100, 'total_tokens': 120, 'completion_time': 0.016666667, 'prompt_time': 0.012911195, 'queue_time': 0.021852961999999997, 'total_time': 0.029577862}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None} id='run-55ef8edc-2e9a-450b-bde0-61f52162c314-0' usage_metadata={'input_tokens': 100, 'output_tokens': 20, 'total_tokens': 120}
